To Do:
- Make List page have subpages (e.g., list of forms, list of forms using a field, see below), make these fileres visible after the intro paragraph
- Make list of forms using a given field with description with embeded question bank info

Thought: make a pdf editor the first step in the weaver. Leverage the name normalizer but allow folks to add and remove fields and rename. 

In [436]:
import pandas as pd
import re
import PyPDF2
import os
from os import walk
import os.path
from os import path
import shutil

import time
from datetime import date

from joblib import dump, load

In [437]:
jurs = [
        #["CA","California"],
        ["ME","Maine"],["MI","Michigan"],["UT","Utah"]]
default_jur = "MI"

today = date.today().strftime("%Y-%m-%d")

In [438]:
def removeSpecial(text):
    text = re.sub('[^a-zA-Z0-9]',"_",text)
    return re.sub('_+',"_",text)

In [439]:
included_fields = load('../data/processed/ML/norm_fields/included_fields.joblib') 

files_df = pd.read_csv("../data/processed/form_data.csv")

print(len(files_df))
display(files_df.head())

files_sim_df = pd.read_csv("../data/processed/form_sim_data.csv")

print(len(files_sim_df))
files_sim_df.head()

1746


,id,jurisdiction,source,title,group,url,filename,downloaded,pages,fields,fields_conf,fields_old,f_per_p,reading
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-11,2.0,"['name__1', 'name__2', '*docket_number', '*use...","[0.61, 0.65, 1.0, 1.0, 1.0, 0.61, 0.53, 0.57, ...","['NAME 1', 'NAME 2', 'CASE NUMBER', 'ADDRESS',...",7.500000,10.0
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-11,2.0,"['*docket_number', '*users1_birthdate', 'male'...","[1.0, 1.0, 0.7, 0.61, 1.0, 1.0, 1.0, 1.0, 0.71...","['Court Case', 'Date of Birth', 'Male', 'Juven...",19.000000,11.0
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-11,5.0,"['probation_department_court', 'special_condit...","[0.73, 0.65, 0.7, 0.54, 0.68, 0.67, 0.62, 0.59...",['of the probation department or by order of t...,4.800000,15.0
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-11,3.0,"['*users1_name', '*users1_address_line_one', '...","[1.0, 1.0, 1.0, 1.0, 0.61, 0.7, 0.67, 1.0, 0.6...","['Name', 'Address', 'City State Zip', 'Phone',...",14.666667,10.0
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-11,1.0,"['*users1_name', '*users1_address_line_one', '...","[1.0, 1.0, 1.0, 1.0, 0.61, 0.7, 0.67, 1.0, 0.6...","['Name', 'Address', 'City State Zip', 'Phone',...",19.000000,11.0


1190282


,id_1,id_2,title_2,similarity
0,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,eb4e35c3b9f355d63ad14ae71866d44b14e61196763553...,Answer to Complaint to Recover Possession of P...,0.979167
1,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,74723a905136a3e30a9eec9aa9191f5d3e4b800ae319df...,"Answer, Damage/Health Hazard to Property, Land...",0.937500
2,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,ee10d4fd30c1faf309bd7b1a91da0794ec8862ee91c355...,"Answer, Damage/Health Hazard to Property, Land...",0.918367
3,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,5345e31f71a6ac7dc730e96039d35c9fc5d2cda658332e...,"Answer, Nonpayment of Rent, Landlord-Tenant",0.918367
4,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,98d5e7eedefb48b57a67dcbf928893c8032c1dec85e867...,"Answer, Nonpayment of Rent, Landlord-Tenant",0.937500


In [440]:
csv_df = files_df.copy()
csv_df["new_url"] = "https://courtformsonline.org/forms/e12318ce9037e92ee8e48963f550e1a3229a17a2437b9ebc3f29c30b.pdf"+csv_df["id"]+".pdf"
csv_df = csv_df.rename(columns={"reading":"reading_level","fields": "standardized_fields","url":"original_url","fields_old":"original_fields","downloaded":"downloaded_on"})
for state in csv_df["jurisdiction"].unique():
    tmp_df = csv_df[csv_df["jurisdiction"]==state][["title","group","pages","reading_level","downloaded_on","original_url","original_fields","new_url","standardized_fields","jurisdiction"]]
    #display(tmp_df)
    tmp_df.to_csv("docs/list/%s/%s_form_data.csv"%(state,state), index=False, encoding="utf-8")    

In [441]:
def clean_n(n):
    if pd.notnull(n):
        return int(n)
    else:
        return 0

In [442]:
def header(title,description,path="./",wide=0):
    
    if wide==1:
        pclass = "content_wide"
    else:
        pclass = "content"
    
    html = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<HTML xmlns="http://www.w3.org/1999/xhtml"
      xmlns:og="http://ogp.me/ns#"
      xmlns:fb="http://www.facebook.com/2008/fbml">
<HEAD>
    <title>%s</title>
    <!-- Global site tag (gtag.js) - Google Analytics -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=UA-108858221-1"></script>
    <script>
      window.dataLayer = window.dataLayer || [];
      function gtag(){dataLayer.push(arguments);}
      gtag('js', new Date());

      gtag('config', 'UA-108858221-1');
    </script>
    <meta http-equiv="Content-type" content="text/html;charset=UTF-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" />
    <meta name="apple-mobile-web-app-capable" content="no" />
    <link rel="apple-touch-icon" href="%simages/icon_300.png"/>
    <meta property="og:type" content="website"/>
    <meta property="og:title" content="%s"/>
    <meta property="og:description" content="%s"/>
    <meta property="og:image" content="%simages/thefuture.png"/>

    <meta name="twitter:card" content="summary_large_image">
    <meta name="twitter:site" content="@SuffolkLITLab">
    <meta name="twitter:creator" content="@SuffolkLITLab">
    <meta name="twitter:title" content="%s">
    <meta name="twitter:description" content="%s">
    <meta name="twitter:image" content="http://www.davidcolarusso.com/images/thefuture.png"/>

    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico">
    <link rel="apple-touch-icon" href="%simages/icon.png"/>
    <link rel="stylesheet" href="https://code.jquery.com/ui/1.11.1/themes/smoothness/jquery-ui.css">
    <link rel="stylesheet" type="text/css" href="%scss/style.css?v=%s">
    <script src="https://code.jquery.com/jquery-1.11.1.min.js"></script>
    <script src="https://code.jquery.com/jquery-1.10.2.js"></script>
    <script src="https://code.jquery.com/ui/1.11.1/jquery-ui.js"></script>

</HEAD>
<BODY BGCOLOR="#ffffff" BACKGROUND="" MARGINWIDTH="0" MARGINHEIGHT="0">
<div style="box-sizing: border-box;width:100%%;background:#d6392b;color:white;padding:15px;text-align:center;">
<b>This tool is in alpha</b>, and this page is just for testing purposes. <b>DO NOT RELY ON THE CONTENT FOUND HERE!</b></div>
<div class="%s">

    <div id="icon" style="background-size: 100px 100px;background-image: url('%simages/seal.jpg');"><a href="%s" alt="home"><img src="%simages/space.gif" width="100px" height="100px;" border="0" alt="LIT Logo"/></a></div>
    <h1 style="text-align:center;">
        The Legal Innovation & Technology Lab's Form Explorer <sup><font size=+1>Beta</font></sup>
        <center style="margin-top:5px;">
          <span class="subtitle">@ Suffolk Law School&nbsp;<font style="font-weight:normal">Last updated %s</font></span>
        </center>
    </h1>
"""%(title,path,title,description,path,title,description,path,path,today,pclass,path,path,path,today)
    return html
    
def footer(path="./"):
    html = """

<div class="footer">
    <a href="/" alt="home"><img src="%simages/blue_logo.png" width="50px" align="left" border="0" alt="LIT Logo"/></a>
<font size=-1><a href="mailto:litlab@suffolk.edu">Email</a> | <a href="https://github.com/SuffolkLITLab" target="_blank">GitHub</a> | <a href="/terms">Terms &amp; Privacy</a> | <a href="/credits">Credits</a></font>
</div>

</BODY>
</HTML>
"""%(path)
    
    return html

# Home page

In [443]:
def home():
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="." class="menu" style="color:black;">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp;
      <a href="list/" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="sim/" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="flow/" class="menu">&nbsp;Flows&nbsp;</a>
      
        </p>
    </div>

  <div class="content"><h2><a name="title" href="#title" class="anchor"></a>What is the Form Explorer?</h2>

    <div style="float:left;width:100%">
      <div class="r_img_embed" style="max-width:500px;">
        <img src="images//bigdata.png" alt="Image of the character Data as a giant standing next to the Supreme Court" width="100%"/>
        <div class="caption">Big Data &amp; the Law, h/t <a href="https://flic.kr/p/7YZubW" target="_blank">Tim Sackton</a> &amp; <a href="https://medium.com/@wtrsld/big-data-made-me-do-it-5bfc3f46871c" target="_blank">Josh Lee</a></div>
      </div>
      <p>
        Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nullam non erat quis augue mollis convallis. Aliquam eu varius massa. Nam convallis lacinia magna, eget aliquet orci congue at. Interdum et malesuada fames ac ante ipsum primis in faucibus. Curabitur eu nisl eu libero porttitor convallis quis ut est. Duis porttitor turpis erat, ut viverra metus fringilla sit amet. Vestibulum tincidunt nibh id libero euismod, sed congue diam lobortis. Vestibulum maximus posuere odio, eget placerat nunc efficitur quis. Proin ut odio sem. Phasellus interdum massa sed sodales eleifend. Cras condimentum, risus et volutpat tincidunt, elit justo vulputate lacus, eget hendrerit purus turpis in nisl. Fusce placerat ex ac hendrerit condimentum. Pellentesque eget auctor leo.
      </p><p>
        Vivamus porttitor posuere sodales. Maecenas id pharetra diam. Nam quis erat quis enim placerat congue et non dui. Donec tempor vehicula ante eget tempor. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Sed vitae orci neque. Maecenas ultricies dignissim neque quis consectetur. In sit amet luctus erat. Pellentesque ultrices sed dui nec feugiat. In nunc ante, bibendum non molestie imperdiet, molestie vel massa. Vestibulum a dictum orci, fringilla rutrum quam. Cras pellentesque congue porttitor. Ut neque felis, tincidunt euismod porttitor sed, feugiat et ligula. Duis vehicula eleifend elit sit amet tempus. Vestibulum condimentum libero vitae justo accumsan, vel sodales erat mattis.
      </p><p>
        Nam nec turpis elementum, tempus massa eget, dictum dui. Donec rutrum nisi pretium faucibus vestibulum. Mauris in ultrices arcu. Nulla vel sem a velit pretium consequat sed vitae felis. Nunc velit tortor, varius at accumsan id, maximus at velit. In cursus metus a purus condimentum convallis. Donec ullamcorper leo eget massa iaculis vehicula. Fusce bibendum aliquam arcu semper laoreet. Quisque consectetur finibus purus sed faucibus. Sed purus ligula, malesuada et diam quis, tincidunt suscipit enim. Curabitur eget quam a tellus suscipit sollicitudin nec ut arcu. Donec bibendum urna a dapibus porttitor. Aliquam in iaculis orci. Etiam condimentum auctor elit eget aliquam. Nulla facilisi. Proin at vestibulum nunc.
      </p>
      <p>
      We have downloaded and parsed X forms from Y jurisdictions. Use the menu above to explore or download summary data here: <a href="javascript:alert('Coming Soon!')">form_data.csv</a>
      </p>
    </div>
"""   
    return html

In [444]:
html = header("Form Explorer","description goes here")+home()+footer()
text_file = open("docs/index.html", "w")
text_file.write(html)
text_file.close()

# redirects

In [445]:
def redirects(jur):
    html = """<html>
<head>
<META http-equiv="CACHE-CONTROL" CONTENT="NO-CACHE">
<meta http-equiv="refresh" content="0; url=%s" />
</head>
</html>
"""%jur
    return html

In [446]:
html = redirects(default_jur)

text_file = open("docs/sim/index.html", "w")
text_file.write(html)
text_file.close()

text_file = open("docs/flow/index.html", "w")
text_file.write(html)
text_file.close()

text_file = open("docs/list/index.html", "w")
text_file.write(html)
text_file.close()

# Similarity Viewer

In [447]:
def simview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu" style="color:black;">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
      
        </p>
    </div>

    <div class="content_wide"><h2><a name="title" href="#title" class="anchor"></a>Similar Forms View (%s Court Forms)</h2>
        <p>
          Below you can explore the connections between forms. Each form is represented by a circle.  
          The lines between each circle represent shared data fields.
        </p>
        Choose a grouping to explore: 
        <select id="graphselect" onChange="$('#graph').attr('src', 'graphs/'+this.value+'.html');$('#grouplink').attr('href', '../../list/%s/#'+$('#graphselect').val());">
        """%(jur[0],jur[0],jur[0],jur[1],jur[0])
    
    k = 0
    for topic in files_df[files_df["jurisdiction"]==jur[0]]["group"].unique():
        html+="<option value='%s'"%(removeSpecial(topic))
        if k==0:
            html+=" SELECTED"
        html+=">%s</option>"%(topic)
        k+=1
    
    html+="""</select>
        &nbsp;&nbsp;<a href="../../list/%s/#%s" id="grouplink">visit froms in this group</span>
        <!--
            Possible library for viewer:
            https://www.sigmajs.org/
        <iframe src="https://www.sigmajs.org/demo/index.html" width="100%%" height="700px;"></iframe>
        -->
        <iframe id="graph" src="graphs/%s.html" width="100%%" height="635px;" frameBorder="0" onLoad="$('#grouplink').attr('href', '../../list/%s/#'+$('#graphselect').val());"></iframe>

    </div>
"""%(jur[0],removeSpecial(files_df[files_df["jurisdiction"]==jur[0]]["group"].unique()[0]),removeSpecial(files_df[files_df["jurisdiction"]==jur[0]]["group"].unique()[0]),jur[0])
    return html

In [448]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/sim/%s"%(jur[0])):
        os.makedirs("docs/sim/%s"%(jur[0]))
    #else:
    #    shutil.rmtree("docs/sim/%s"%(jur[0]))
    #    os.makedirs("docs/sim/%s"%(jur[0]))
        
    html = header("Form Explorer","description goes here",path,wide=1)+simview(jur,path)+footer(path)
    text_file = open("docs/sim/%s/index.html"%jur[0], "w")
    text_file.write(html)
    text_file.close()

['ME', 'Maine']
['MI', 'Michigan']
['UT', 'Utah']


# Flow View

In [449]:
def flowview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu" style="color:black;">&nbsp;Flows&nbsp;</a>
        </p>
    </div>


      <div class="content_wide"><h2><a name="title" href="#title" class="anchor"></a>User Flow (%s Court Forms)</h2>
        <p style="background:#fcf19d">
        <i><b>Under Construction:</b> With historical form data we could provde insights on a users's flow through forms 
          (e.g., after using form X, 20%% of people used form Y). 
          If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
        <div style="overflow-x: scroll;:hidden;overflow-y: hidden;">
          <img src="../../images/.jpg" alt="" width="1500px"/>
        </div>

      </div>
"""%(jur[0],jur[0],jur[0],jur[1])
    return html

In [450]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/flow/%s"%(jur[0])):
        os.makedirs("docs/flow/%s"%(jur[0]))
    else:
        shutil.rmtree("docs/flow/%s"%(jur[0]))
        os.makedirs("docs/flow/%s"%(jur[0]))
        
    html = header("Form Explorer","description goes here",path,wide=1)+flowview(jur,path)+footer(path)
    text_file = open("docs/flow/%s/index.html"%jur[0], "w")
    text_file.write(html)
    text_file.close()

['ME', 'Maine']
['MI', 'Michigan']
['UT', 'Utah']


# List VIew

In [451]:
def listview(jur,path="./"):
    
    pages = files_df[(files_df["jurisdiction"]==jur[0])]["pages"].mean()
    f_per_p = files_df[(files_df["jurisdiction"]==jur[0])]["f_per_p"].mean()
    fields = files_df[(files_df["jurisdiction"]==jur[0])]["f_per_p"].mean()*files_df[(files_df["jurisdiction"]==jur[0])]["pages"].mean()
    reading = files_df[(files_df["jurisdiction"]==jur[0])]["reading"].mean()
    forms= len(files_df[(files_df["jurisdiction"]==jur[0])])
    
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu" style="color:black;">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
        </p>
    </div>
"""%(jur[0],jur[0],jur[0])
    
    html += """
    <div class="content">
    <div style="float:left;width:60%%;">
    <h2><a name="title" href="#title" class="anchor"></a>List of %s Court Forms</h2>

    <p>
    Below you will find %s forms. The average form is %.0f pages long, with %.0f fields per page. The average form is written at a %.2f-grade reading level.
    </p>
    
    <p>Download a .csv file including meta data on all %s forms, including source urls, field names, and reading levels: 
    <a href="%s_form_data.csv" target="_blank">%s_form_data.csv</a>
    </p>
    
    Choose Ordering: 
    <select id="form_ordering" onChange="sort_links()">
        <option value='court_grouping'>Court Groupings</option>
        <option value='LIST_grouping'>LIST Groupings</option>
        <option value='most_used'>Most Used</option>
    </select>
    <script>
        function sort_links(){
            if($('#form_ordering').val() == "court_grouping"){
                $('#court_grouping').show()
                $('#LIST_grouping').hide()
                $('#most_used').hide()
            } else if($('#form_ordering').val() == "LIST_grouping"){
                $('#court_grouping').hide()
                $('#LIST_grouping').show()
                $('#most_used').hide()
            } else if($('#form_ordering').val() == "most_used"){
                $('#court_grouping').hide()
                $('#LIST_grouping').hide()
                $('#most_used').show()
            }
        }
    
    </script>
    
    <div id="court_grouping" style="width:100;">
    
    <p><i>Links are grouped by categories found on the court's website.</i></p>
    
    """%(jur[1],forms,pages,fields,reading,forms,jur[0],jur[0])

    for cat in files_df[files_df["jurisdiction"]==jur[0]]["group"].unique():
        html += """
        <h3><a name="%s" href="#%s" class="anchor"></a>%s</h3>
        <ul>"""%(removeSpecial(cat),removeSpecial(cat),cat)

        for index,row in files_df[(files_df["jurisdiction"]==jur[0]) & (files_df["group"]==cat)].iterrows():
            #print(row[1][0])
            html += """<li><a href="../../form/%s/%s.html">%s</a></li>"""%(jur[0],row["id"],row["title"])
            
        html += "</ul><p style=\"text-align:center\"><a href=\"#title\">go to top</a></p>"
        
    html+="""</div>
    
    <div id="LIST_grouping" style="display:none;width:100;">
    
    <p><i>Links are grouped by categories from the <a href="https://taxonomy.legal/" target="_blank" class="exlink">Legal Issue Taxonomy (LIST)</a>. Note: the grouping below was doneby machine and may have some errors.</i></p>
    <p style="background:yellow;"><b>Coming Soon!</b></p>
    
    </div>
    
    <div id="most_used" style="display:none;width:100;">
    
    <p><i>Links are orderd by most used.</i></p>
    
    <p style="background:#fcf19d">
        <i>
            <b>Under Construction:</b> With historical form data we could provde insights on how frequently forms are used.
            If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
        </i>
    </p>
    
    </div>
    </div>
    """
    
    html+="""<div style="float:right;width:35%%;">
            <h2>Search %s Forms</h2>
            <table cellpadding="0px"><tr><td width="100%%">
            <input id="q" style="width:100%%">
            </td><td width="1%%">&nbsp;&nbsp;&nbsp;</td><td width="1%%">
            <input type="button" value="Search" onClick="open_search();">
            </td></tr></table>
            <script>
                sort_links();
            
                function open_search(){
                    alert('Coming Soon!');
                }
                function open_search_(){
                    window.open('https://www.google.com/search?q='+$('#q').val()+'+site:suffolklitlab.org/form-explorer/form/%s')
                }
                $("#q").on('keyup', function (e) {
                    if (e.key === 'Enter' || e.keyCode === 13) {
                        open_search()
                    }
                });
            </script>
            <h2>Standard Field Names</h2>
            <p>We maintain a bank of <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables">standard questions and field names</a>. A list of common fields is found below, click a field to see those forms using it.</p>
            <ul>
    """%(jur[1],jur[0])
        
    for field in included_fields:
        print("\t"+field)
        html += """<li><a href="fields/%s.html">%s</a></li>"""%(field,field)
        
    html+="</ul></div>"

    return html

In [452]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/list/%s"%(jur[0])):
        os.makedirs("docs/list/%s"%(jur[0]))
    #else:
    #    shutil.rmtree("docs/list/%s"%(jur[0]))
    #    os.makedirs("docs/list/%s"%(jur[0]))
        
    html = header("Form Explorer","description goes here",path,wide=0)+listview(jur,path)+footer(path)
    text_file = open("docs/list/%s/index.html"%(jur[0]), "w", encoding="utf-8")
    text_file.write(html)
    text_file.close()

['ME', 'Maine']
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
['MI', 'Michigan']
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
['UT', 'Utah']
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	si

# Field Labels

In [453]:
def fields(field_name,jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../../'+this.value+'/fields/%s.html'">"""%field_name
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="../../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
        </p>
    </div>
"""%(jur[0],jur[0],jur[0])
    
    regex = field_name
    forms_with_var = files_df[(files_df["jurisdiction"]==jur[0]) & files_df['fields'].str.contains(regex)]

    html += """
      <div class="content">
      <div style="float:left;width:60%%;">
      <h2><a name="title" href="#title" class="anchor"></a>%s court forms that use: <i>%s</i></h2>
      <p>Out of <a href="../">%s forms</a>, <span style="background:yellow"><b>the following %s forms use <i>%s</i>:</b></span>
      
      To learn more about our standard field names, read <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables">Field labels to use in template files</a>.
      
      </p>
"""%(jur[1],field_name,len(files_df[files_df["jurisdiction"]==jur[0]]),len(forms_with_var),field_name)
    
    for cat in forms_with_var["group"].unique():
        html += """
        <h3><a name="%s" href="#%s" class="anchor"></a>%s</h3>
        <ul>"""%(cat,cat,cat)

        for index,row in forms_with_var[forms_with_var["group"]==cat].iterrows():
            #print(row[1][0])
            html += """<li><a href="../../../form/%s/%s.html">%s</a></li>"""%(jur[0],row["id"],row["title"])
            
        html += "</ul><p style=\"text-align:center\"><a href=\"#title\">go to top</a></p>"
        
    html += "</div>"

    html+="""</div><div style="float:right;width:35%%;">
            <h2>Search %s Forms</h2>
            <table cellpadding="0px"><tr><td width="100%%">
            <input id="q" style="width:100%%">
            </td><td width="1%%">&nbsp;&nbsp;&nbsp;</td><td width="1%%">
            <input type="button" value="Search" onClick="open_search();">
            </td></tr></table>
            <script>
                function open_search(){
                    alert('Coming Soon!');
                }
                function open_search_(){
                    window.open('https://www.google.com/search?q='+$('#q').val()+'+site:suffolklitlab.org/form-explorer/form/%s')
                }
                $("#q").on('keyup', function (e) {
                    if (e.key === 'Enter' || e.keyCode === 13) {
                        open_search()
                    }
                });
            </script>
            <p style="text-align:center;background:#d2e6fa"><a href="../">BACK TO ALL %s FORMS</a></p>
            <h2>Standard Field Names</h2>
            <p>We maintain a bank of <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables">standard questions and field names</a>. A list of common fields is found below, click a field to see those forms using it.</p>
            <ul>
    """%(jur[1],jur[0],jur[1].upper())
        
    for field in included_fields:
        print("\t"+field)
        html += """<li><a href="fields/%s.html">%s</a></li>"""%(field,field)
        
    html+="</ul></div>"    
    
    return html

In [454]:
for jur in jurs:  
    print(jur)
    path = "../../../"
    for field in included_fields:
        print("\t"+field)
        html = header("Form Explorer","description goes here",path)+fields(field,jur,path)+footer(path)
        text_file = open("docs/list/%s/fields/%s.html"%(jur[0],field), "w")
        text_file.write(html)
        text_file.close()

['ME', 'Maine']
	users1_name
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_birthdate
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_address_line_one
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county

	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	docket_number
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	trial_court_county
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	u

# Create form pages

In [461]:
def formview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore: """
    
    for item in jurs: 
        if item[0]==jur[0]:
            this_state = item[1]
            html += "%s"%this_state

    html += """
      </span>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
        </p>
    </div>

"""%(jur[0],jur[0],jur[0])
    
    if jur[0]!="CA":
        linkurl = "https://courtformsonline.org/forms/%s.pdf"%row["id"]
        iframelink = linkurl
    else:
        linkurl = "javascript:alert('Califonia machine-processed forms are not yet available.')''"
        iframelink = row["url"]
        
    html += """
  <div class="content_wide" style="padding-left:0px;">
    <div class="pdf">
      <iframe src="%s" width="100%%" height="1100px;"></iframe>
    </div>
    <div class="pdfinfo">
      <h1><a name="title" href="#title" class="anchor"></a>%s</h1>
      <p>
        This info page is part of the LIT Lab's <i>Form Explorer</i> project. It is not associated with the %s state courts. 
        To learn more about the project, check out our <a href="../../">about page</a>.
      </p>
      <p>
        <b>Downloads:</b> In order to get the data on this page we scan forms for... You can download both the <a href="%s" target="_blank" class="exlink">original form</a> (last checked on %s) 
        and the <a href="%s">machine-processed form</a> with normalized data fields.
      </p>
      <h3><a name="about_form" href="#about_form" class="anchor"></a>About This Form:</h3>
      <ul>"""%(iframelink,row["title"],this_state,row["url"],row["downloaded"],linkurl)
    
    html += """
        <li>Sourced from <a href="http://%s" target="_blank" class="exlink">%s</a> on %s</li>
        <li>Court Grouping: <a href="../../list/%s/#%s">%s</a></li>
        <li><a href="https://taxonomy.legal/" target="_blank" class="exlink">LIST</a> Grouping(s): <span style="background:yellow;">Coming Soon!</span></li>
        <li>Page(s): %s </li>
        <li>Fields(s): %s </li>
        <li>Average fields per page: %s</li>
        <li>Reading Level: Grade %s</li>
      </ul>
      
      <h3><a name="about_users" href="#about_users" class="anchor"></a>About Users of This Form:<sup>&dagger;</sup></h3>
      <p style="background:#fcf19d"><sup>&dagger;</sup> <i>With historical form data we could provde insights on users of this for  
      (e.g., filers of this form file an average of <b>X</b> other forms. <b>Y%%</b> of them identify as male...).  
      If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
      
      <h3><a name="fields" href="#fields" class="anchor"></a>Identified Data Fields and Utilization:<sup>*</sup></h3>
      <p>We have done our best to automaticly identify and name form fields according to our <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables/">naming conventions</a>. 
      When possible, we've used names tied to our question library. See e.g., <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/question_library/names">user1_name</a>.
      If we think we've found a match to a question in our library, it is highlighted in green. The new names found here are auto generated. So, you will probably need to edit some of them.</p>
      
      <p style="background:#fcf19d"><sup>*</sup><i> With historical form data we could provde insights on field utilization  
      (e.g., <b>X%%</b> of the time this field is left blank...). 
       If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
      
      <ul>"""%(row["source"],row["source"],row["downloaded"],jur[0],removeSpecial(row["group"]),row["group"],clean_n(row["pages"]),clean_n(row["pages"]*row["f_per_p"]),clean_n(row["f_per_p"]),clean_n(row["reading"]))
    
    try:
        j = 0
        for f in eval(row["fields"]):
            html += "<li>"
            if re.search('^\*',f):
                f = re.sub("^\*","",f)
                f_link = re.sub("__\d+$","",f)
                #v = re.search('^(\w*)',f).groups()[0]
                #f = re.sub("\|","</code></a>",f)
                html +="<code style=\"background:#a2e874\"><a href=\"../../list/%s/fields/%s.html\">%s</a></code> was <i>%s</i> (%.2f conf)<!--100%% completed. <a href=\"\">Learn more about distribution of answers.</a>--></li>"%(jur[0],f_link,f,eval(row["fields_old"])[j],eval(row["fields_conf"])[j])   
            else: 
                #v = re.search('^(\w*)',f).groups()[0]
                #f = re.sub("\|","</code>",f)
                html +="<code>%s</code> was <i>%s</i> (%.2f conf)<!--100%% completed. <a href=\"\">Learn more about distribution of answers.</a>--></li>"%(f,eval(row["fields_old"])[j],eval(row["fields_conf"])[j])
            j+=1    
    except:
        print("")

    if not pd.isnull(row["fields"]):
        if len(eval(row["fields"]))==0:
            html+="<li>No fields found</li>"
    else:
            html+="<li>No fields found</li>"
        
    html += """
        <!--<li><a href="../form/">show more</a></li>-->
        </ul>

      </ul>"""
    
    sim_forms = files_sim_df[files_sim_df["id_1"]==row["id"]].sort_values(by=['similarity'], ascending=False)[:5]
    
    html+="""      <h3><a name="similar" href="#similar" class="anchor"></a>Similar Forms:</h3>
      <p>For more context, visit the <a href="../../sim/%s">Similar Forms View</a>.</p>
      <ul>"""%jur[0]

    for index_2,row_2 in sim_forms.iterrows():
        html+="<li><a href=\"%s.html\">%s</a> (shares %s%% of data fields)</li>"%(row_2["id_2"],row_2["title_2"],clean_n(row_2["similarity"]*100))

    if len(sim_forms)==0:
        html+="<li>No matches found</li>"
        
    html+="""</ul>
      <!--
      <li><a href="../form/">show more</a></li>-->"""
        
    html += """
      
      <h3><a name="before" href="#before" class="anchor"></a>Before using this form, users used:<sup>&dagger;&dagger;</sup></h3>
      <p>For more context, visit the <a href="../../flow/%s">Flows View</a>.</p>
       <p style="background:#fcf19d"><sup>&dagger;&dagger;</sup><i> With historical form data we could provde insights on 
       what forms a user used before this one. 
       If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
      
      <h3><a name="after" href="#after" class="anchor"></a>After using this form, users used:<sup>**</sup></h3>
      <p>For more context, visit the <a href="../../flow/%s">Flows View</a>.</p>
      <p style="background:#fcf19d"><sup>**</sup><i> With historical form data we could provde insights on 
      what forms a user used after this one. 
      If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
      
      <h3><a name="linter" href="#linter" class="anchor"></a>See Suggestions for Improving this Form:</h3>
      <p>Using the Document Assembly Line Linter you can...</p>
      <p>
        <a href="javascript:alert('Coming Soon!');">Run this form through the Linter</a>
      </p>
      
      <h3><a name="weaver" href="#weaver" class="anchor"></a>Create an Interactive Version of this Form:</h3>
      <p>
      The Weaver creates a draft guided interview from a template form, like the one provided here. You can use the link below to open this form in the Weaver.
      To learn more, read <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/generating_code/">"Weaving" your form into a draft interview</a>.
      </p>"""%(jur[0],jur[0])
    
    if jur[0]!="CA":
        html+="""
      <p>
        <a href="https://apps-dev.suffolklitlab.org/start/ALWeaver/assembly_line?new_session=1&form_to_use=https://courtformsonline.org/forms/%s.pdf" target="_blank" class="launch"><div class="launch">Launch Form in Weaver</div></a>
      </p>
      """%(row["id"])
    else:
        html+="""
      <p>
        <a href="javascript:alert('Not available for California Forms at this time.')" target="_blank" class="launch"><div class="launch">Launch Form in Weaver</div></a>
      </p>
      """
            
    html+="""
    </div>
  </div>
    """
    return html

In [ ]:
path = "../../"

for jur in jurs:    
    if not os.path.exists("docs/form/%s"%(jur[0])):
        os.makedirs("docs/form/%s"%(jur[0]))
    else:
        shutil.rmtree("docs/form/%s"%(jur[0]))
        os.makedirs("docs/form/%s"%(jur[0]))
        

for index,row in files_df.iterrows():
    print(row["jurisdiction"],row["title"])
    
    html = header(row["jurisdiction"]+": "+row["title"],"description goes here",path,wide=1)+formview([row["jurisdiction"],""],path)+footer(path)
    text_file = open("docs/form/%s/%s.html"%(row["jurisdiction"],row["id"]), "w", encoding="utf-8")
    
    text_file.write(html)
    text_file.close()

UT Community Service Worksheet Third District Juvenile Court  
UT Third District Juvenile Court Work Program Referral 
UT Utah State District Juvenile Court Probation Order 
UT Adoptee's Consent to Adoption and Waiver of Rights 
UT Adoption Agreement 
UT Adoption Decree 
UT Birth Mother's Consent to Adoption and Waiver of Rights 
UT Certificate of Readiness for Adoption Hearing 
UT Ex Parte Motion for Alternative Service 
UT Findings of Fact and Conclusions of Law 
UT Findings of Fact, Conclusions of Law, and Order on Motion for Alternative Service 
UT Guardian's Consent to Adoption and Waiver of Notice 
UT Non-custodial Father's Consent to Adoption and Waiver of Rights 
UT Notice of Adoption and Notice of Rights for Publication 
UT Notice of Petition to Adopt and Notice of Rights 
UT Notice of Petition to Adopt and Notice of Rights 
UT Petition to Adopt a Minor Stepchild 
UT Proof of Alternative Service 
UT Proof of Completed Service 
UT Request to Submit for Decision 
UT Utah Distric

In [ ]:
files_sim_df[files_sim_df["similarity"]!=0]["similarity"].median()